# TMDB API로 데이터 수집

## TMDB TV Series 데이터 전수 수집

1. 수집 대상: TMDB TV Series & Seasons 데이터
2. 수집 기간: 2016년 1월 1일 ~ 2025년 11월 30일 (약 11년치)
3. 수집 범위: TV Series과 Seasons 전체 페이지 (에피소드 제외)
4. 수집 방식: 100% 비동기(Pure Async) 방식
5. 수집 컬럼: 시리즈 50개, 시즌 14개
6. 수집 조건: 페이지 누락 0%, TMDB API 초당 요청 수(40 req/s) 제한 대응, 500페이지 제한 대응
7. 수집 결과: 총 개 데이터, 약 분 소요 (시리즈 개 컬럼 / 시즌 개 컬럼)

In [3]:
# ==========================================================
# TMDB TV SERIES + SEASONS FULL ASYNC COLLECTOR
# Series 51 cols / Seasons 15 cols (FINAL + poster_path)
# ==========================================================

import os
import asyncio
import aiohttp
import async_timeout
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv

# ==========================================================
# Jupyter async 지원
# ==========================================================
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("✅ nest_asyncio 적용 완료")
except:
    pass

# ==========================================================
# 기본 설정
# ==========================================================
load_dotenv()
API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise ValueError("TMDB_API_KEY missing")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS = {"accept": "application/json"}
MAX_CALLS_PER_SECOND = 38
TIMEOUT = 10
MAX_RETRIES = 5

START_DATE = "2016-01-01"
END_DATE = "2025-11-30"

# temp
SERIES_TEMP = Path("tv_series_temp.csv")
SEASONS_TEMP = Path("tv_seasons_temp.csv")

# final
SERIES_CSV = "tv_series_2016_2025_FULL.csv"
SEASONS_CSV = "tv_seasons_2016_2025_FULL.csv"
SERIES_PARQ = "tv_series_2016_2025_FULL.parquet"
SEASONS_PARQ = "tv_seasons_2016_2025_FULL.parquet"

# ==========================================================
# Rate limiter
# ==========================================================
class AsyncRateLimiter:
    def __init__(self, max_calls, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock = asyncio.Lock()

    async def acquire(self):
        async with self.lock:
            now = asyncio.get_event_loop().time()

            # 기간 밖 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            # 초당 호출 수 초과 시 sleep
            if len(self.calls) >= self.max_calls:
                sleep_for = self.period - (now - self.calls[0]) + 0.01
                await asyncio.sleep(max(0, sleep_for))

                now = asyncio.get_event_loop().time()
                while self.calls and now - self.calls[0] > self.period:
                    self.calls.popleft()

            self.calls.append(now)

rate_limiter = AsyncRateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# TMDB GET
# ==========================================================
async def tmdb_get(session, path, params=None, retry=0):
    if retry >= MAX_RETRIES:
        return None

    params = params or {}
    params.setdefault("api_key", API_KEY)
    params.setdefault("language", "en-US")

    url = f"{BASE_URL}{path}"

    await rate_limiter.acquire()

    try:
        async with async_timeout.timeout(TIMEOUT):
            async with session.get(url, params=params, headers=HEADERS) as resp:
                # 레이트 리밋
                if resp.status == 429:
                    wait = float(resp.headers.get("Retry-After", 2)) * (2 ** retry)
                    await asyncio.sleep(wait)
                    return await tmdb_get(session, path, params, retry + 1)

                # Not found
                if resp.status == 404:
                    return None

                # 서버 에러
                if 500 <= resp.status < 600:
                    await asyncio.sleep(2 ** retry)
                    return await tmdb_get(session, path, params, retry + 1)

                resp.raise_for_status()
                return await resp.json()

    except:
        await asyncio.sleep(2 ** retry)
        return await tmdb_get(session, path, params, retry + 1)

# ==========================================================
# discover page
# ==========================================================
async def fetch_discover_page(session, page, gte, lte):
    params = {
        "sort_by": "popularity.desc",
        "first_air_date.gte": gte,
        "first_air_date.lte": lte,
        "page": page,
        "include_adult": "true",   # 🔥 FIX: 문자열 true (TMDB discover 규격)
    }
    data = await tmdb_get(session, "/discover/tv", params)
    if not data:
        return [], 1, 0
    return data.get("results", []), data.get("total_pages", 1), data.get("total_results", 0)

# ==========================================================
# 분할 수집 (500 page limit 회피)
# ==========================================================
async def collect_ids_in_range(session, start, end, depth=0):
    indent = "  " * depth

    res1, total_pages, total_results = await fetch_discover_page(session, 1, start, end)

    if total_results == 0:
        return set()

    # 500페이지 이하면 그대로 전수 수집
    if total_pages <= 500:
        ids = {r["id"] for r in res1}
        if total_pages > 1:
            tasks = [
                fetch_discover_page(session, p, start, end)
                for p in range(2, total_pages + 1)
            ]
            for coro in asyncio.as_completed(tasks):
                r, _, _ = await coro
                for x in r:
                    ids.add(x["id"])
        return ids

    # 500 페이지 초과 → 날짜 반으로 분할
    start_dt = datetime.strptime(start, "%Y-%m-%d")
    end_dt = datetime.strptime(end, "%Y-%m-%d")
    mid_dt = start_dt + (end_dt - start_dt) / 2
    mid = mid_dt.strftime("%Y-%m-%d")
    right_start = (mid_dt + timedelta(days=1)).strftime("%Y-%m-%d")

    left = await collect_ids_in_range(session, start, mid, depth + 1)
    right = await collect_ids_in_range(session, right_start, end, depth + 1)
    return left | right

# ==========================================================
# helpers
# ==========================================================
def list_to_str(lst, key="name"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key, "")) for i in lst if i.get(key))

def list_ids_to_str(lst, key="id"):
    if not lst:
        return ""
    return ", ".join(str(i.get(key)) for i in lst if i.get(key))

# ==========================================================
# providers
# ==========================================================
async def fetch_providers(session, sid):
    data = await tmdb_get(session, f"/tv/{sid}/watch/providers")
    if not data:
        return ""
    out = set()
    for cc in ["US", "GB", "KR"]:
        if cc in data.get("results", {}):
            p = data["results"][cc].get("flatrate", [])
            for x in p:
                if x.get("provider_name"):
                    out.add(x["provider_name"])
    return ", ".join(sorted(out))

# ==========================================================
# single season (시즌 1개 수집)
# ==========================================================
async def fetch_single_season(session, sid, sname, net_names, net_ids, meta):
    sn = meta.get("season_number")
    if sn is None:
        return None

    data = await tmdb_get(session, f"/tv/{sid}/season/{sn}")

    # 시즌 상세 못 가져오면 meta 기반으로 fallback
    if not data:
        return {
            "_id": f"{sid}_{sn}",
            "season_id": meta.get("id"),
            "series_id": sid,
            "series_name": sname,
            "season_number": sn,
            "name": meta.get("name"),
            "air_date": meta.get("air_date"),
            "overview": meta.get("overview"),
            "vote_average": meta.get("vote_average"),
            "vote_count": meta.get("vote_count"),
            "network_names": net_names,
            "network_ids": net_ids,
            "total_episodes": meta.get("episode_count"),
            "avg_episode_runtime": None,
            "poster_path": meta.get("poster_path"),   # 🔥 시즌 포스터 (meta 기준)
        }

    eps = data.get("episodes") or []
    runtimes = [ep.get("runtime") for ep in eps if isinstance(ep.get("runtime"), (int, float))]
    avg_rt = sum(runtimes) / len(runtimes) if runtimes else None

    return {
        "_id": data.get("_id") or f"{sid}_{sn}",
        "season_id": data.get("id"),
        "series_id": sid,
        "series_name": sname,
        "season_number": data.get("season_number"),
        "name": data.get("name"),
        "air_date": data.get("air_date"),
        "overview": data.get("overview"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
        "network_names": net_names,
        "network_ids": net_ids,
        "total_episodes": len(eps),
        "avg_episode_runtime": avg_rt,
        "poster_path": data.get("poster_path"),   # 🔥 시즌 포스터 (상세 기준)
    }

# ==========================================================
# TV 상세 + 시즌 (Series 51 cols / Seasons 15 cols)
# ==========================================================
async def fetch_tv_details_and_seasons(session, sid):
    params = {"append_to_response": "reviews,keywords,aggregate_credits"}
    data = await tmdb_get(session, f"/tv/{sid}", params)
    if not data:
        return None, []

    genres = data.get("genres") or []
    networks = data.get("networks") or []
    last_ep = data.get("last_episode_to_air") or {}

    net_names = list_to_str(networks)
    net_ids = list_ids_to_str(networks)

    # ========== SERIES 51개 ==========
    series = {
        "id": data.get("id"),
        "title": data.get("name"),            # 🔥 name 필드는 여기만 사용 (컬럼엔 'title')
        "type": "tv_series",
        "adult": data.get("adult"),
        "backdrop_path": data.get("backdrop_path"),
        "created_by": list_to_str(data.get("created_by")),
        "episode_run_time": (
            ", ".join(map(str, data.get("episode_run_time", [])))
            if data.get("episode_run_time") else ""
        ),
        "first_air_date": data.get("first_air_date"),
        "genres": list_to_str(genres),
        "genre_ids": ", ".join(str(g.get("id")) for g in genres if g.get("id")),
        "homepage": data.get("homepage"),
        "in_production": data.get("in_production"),
        "languages": ", ".join(data.get("languages", [])) if data.get("languages") else "",
        "last_air_date": data.get("last_air_date"),

        "last_episode_to_air_id": last_ep.get("id"),
        "last_episode_to_air_name": last_ep.get("name"),
        "last_episode_to_air_overview": last_ep.get("overview"),
        "last_episode_to_air_vote_average": last_ep.get("vote_average"),
        "last_episode_to_air_vote_count": last_ep.get("vote_count"),
        "last_episode_to_air_air_date": last_ep.get("air_date"),
        "last_episode_to_air_episode_number": last_ep.get("episode_number"),
        "last_episode_to_air_production_code": last_ep.get("production_code"),
        "last_episode_to_air_runtime": last_ep.get("runtime"),
        "last_episode_to_air_season_number": last_ep.get("season_number"),
        "last_episode_to_air_show_id": last_ep.get("show_id"),
        "last_episode_to_air_still_path": last_ep.get("still_path"),

        "next_episode_to_air": "" if not data.get("next_episode_to_air") else str(data.get("next_episode_to_air")),
        "networks": net_names,
        "number_of_episodes": data.get("number_of_episodes"),
        "number_of_seasons": data.get("number_of_seasons"),

        "origin_country": ", ".join(data.get("origin_country", [])) if data.get("origin_country") else "",
        "original_language": data.get("original_language"),
        "original_name": data.get("original_name"),
        "overview": data.get("overview"),
        "popularity": data.get("popularity"),
        "poster_path": data.get("poster_path"),   # 🔥 시리즈 포스터

        "production_companies": list_to_str(data.get("production_companies")),
        "production_countries": list_to_str(data.get("production_countries")),

        "seasons": "; ".join(
            f"S{s.get('season_number')}: {s.get('name')} ({s.get('episode_count')} eps)"
            for s in (data.get("seasons") or [])
            if s.get("season_number") is not None
        ),

        "spoken_languages": list_to_str(data.get("spoken_languages")),
        "status": data.get("status"),
        "tagline": data.get("tagline"),
        "type_detail": data.get("type"),
        "vote_average": data.get("vote_average"),
        "vote_count": data.get("vote_count"),
    }

    # 리뷰
    rv_block = data.get("reviews", {})
    rv_items = rv_block.get("results", []) if isinstance(rv_block, dict) else []
    rev = []
    for r in rv_items[:5]:
        author = r.get("author", "")
        rating = r.get("author_details", {}).get("rating")
        rt = f"({rating})" if rating is not None else ""
        cont = (r.get("content") or "").replace("\n", " ").replace("\r", " ")[:200]
        rev.append(f"{author}{rt}: {cont}")
    series["review"] = " || ".join(rev)

    # 키워드
    kw_block = data.get("keywords", {})
    kw_items = kw_block.get("results", []) if isinstance(kw_block, dict) else []
    series["keyword"] = ", ".join([k.get("name") for k in kw_items[:20] if k.get("name")])

    # top cast & crew
    credits = data.get("aggregate_credits", {})
    cast = credits.get("cast") or []
    crew = credits.get("crew") or []

    series["top_cast"] = ", ".join([c.get("name") for c in cast[:4] if c.get("name")])

    dirs = set()
    wrs = set()
    for c in crew:
        nm = c.get("name")
        if not nm:
            continue
        for job in (c.get("jobs") or []):
            jn = job.get("job", "")
            if "Director" in jn:
                dirs.add(nm)
            if jn in ["Writer", "Screenplay", "Story"]:
                wrs.add(nm)

    series["directors"] = ", ".join(sorted(dirs)[:10])
    series["writers"] = ", ".join(sorted(wrs)[:10])

    # providers
    series["providers"] = await fetch_providers(session, sid)

    # 시즌 리스트
    seasons_meta = data.get("seasons") or []
    tasks = [
        fetch_single_season(session, sid, data.get("name") or "", net_names, net_ids, s)
        for s in seasons_meta
        if s.get("season_number") is not None
    ]

    season_records = []
    for coro in asyncio.as_completed(tasks):
        x = await coro
        if x:
            season_records.append(x)

    return series, season_records

# ==========================================================
# 시즌 기반 결측치
# ==========================================================
def fill_series_gaps(series_df, seasons_df):
    print("\n📊 시즌 기반 결측치 보완 시작")

    # episode_run_time
    rtmap = seasons_df.groupby("series_id")["avg_episode_runtime"].mean().round()
    mask = series_df["episode_run_time"].astype(str).str.strip().isin(["", "nan"])
    series_df.loc[mask, "episode_run_time"] = (
        series_df.loc[mask, "id"].map(rtmap).fillna(0).astype(int).astype(str)
    )

    # number_of_episodes
    epmap = seasons_df.groupby("series_id")["total_episodes"].sum()
    mask2 = series_df["number_of_episodes"].isna()
    series_df.loc[mask2, "number_of_episodes"] = (
        series_df.loc[mask2, "id"].map(epmap)
    )

    # number_of_seasons
    seasmap = seasons_df.groupby("series_id")["season_number"].nunique()
    mask3 = series_df["number_of_seasons"].isna()
    series_df.loc[mask3, "number_of_seasons"] = (
        series_df.loc[mask3, "id"].map(seasmap)
    )

    # last_air_date
    lastair = seasons_df.sort_values("air_date").groupby("series_id")["air_date"].last()
    ladmask = series_df["last_air_date"].astype(str).str.strip().isin(["", "nan", "NaT"])
    series_df.loc[ladmask, "last_air_date"] = series_df.loc[ladmask, "id"].map(lastair)

    print("✅ 보완 완료\n")
    return series_df

# ==========================================================
# MAIN
# ==========================================================
async def main():
    print("=" * 90)
    print("🚀 TMDB FULL SERIES + SEASONS COLLECTOR (FINAL + poster_path)")
    print("=" * 90)
    t0 = datetime.now()

    # 기존 temp 삭제
    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    async with aiohttp.ClientSession() as session:
        # 1) ID 수집
        print("\n📌 1단계: ID 수집")
        ids = sorted(list(await collect_ids_in_range(session, START_DATE, END_DATE)))
        print(f"✨ 총 ID: {len(ids):,}개")

        # 2) 상세 + 시즌 수집
        print("\n📌 2단계: 상세 + 시즌 수집")
        batch_size = 500
        sem = asyncio.Semaphore(25)

        async def fetch_one(i):
            async with sem:
                return await fetch_tv_details_and_seasons(session, i)

        processed = 0
        for start in range(0, len(ids), batch_size):
            part = ids[start:start+batch_size]
            tasks = [fetch_one(x) for x in part]

            bs = []
            bse = []

            for coro in asyncio.as_completed(tasks):
                s, ss = await coro
                if s:
                    bs.append(s)
                if ss:
                    bse.extend(ss)

                processed += 1
                if processed % 100 == 0:
                    print(f"⏱ {processed:,}/{len(ids):,}")

            # flush
            if bs:
                pd.DataFrame(bs).to_csv(
                    SERIES_TEMP, mode="a", header=not SERIES_TEMP.exists(),
                    index=False, encoding="utf-8-sig"
                )
            if bse:
                pd.DataFrame(bse).to_csv(
                    SEASONS_TEMP, mode="a", header=not SEASONS_TEMP.exists(),
                    index=False, encoding="utf-8-sig"
                )

    # ==========================================================
    # 3) temp → final DF
    # ==========================================================
    df_series = pd.read_csv(SERIES_TEMP)
    df_seasons = pd.read_csv(SEASONS_TEMP) if SEASONS_TEMP.exists() else pd.DataFrame()

    SERIES_COLS = [
        "id","title","type","adult","backdrop_path","created_by","episode_run_time",
        "first_air_date","genres","genre_ids","homepage","in_production","languages",
        "last_air_date","last_episode_to_air_id","last_episode_to_air_name",
        "last_episode_to_air_overview","last_episode_to_air_vote_average",
        "last_episode_to_air_vote_count","last_episode_to_air_air_date",
        "last_episode_to_air_episode_number","last_episode_to_air_production_code",
        "last_episode_to_air_runtime","last_episode_to_air_season_number",
        "last_episode_to_air_show_id","last_episode_to_air_still_path",
        "next_episode_to_air","networks","number_of_episodes","number_of_seasons",
        "origin_country","original_language","original_name","overview","popularity",
        "poster_path",                              # 🔥 Series poster_path
        "production_companies","production_countries","seasons","spoken_languages",
        "status","tagline","type_detail","vote_average","vote_count","review",
        "keyword","top_cast","directors","writers","providers"
    ]

    SEASON_COLS = [
        "_id","season_id","series_id","series_name","season_number","name","air_date",
        "overview","vote_average","vote_count","network_names","network_ids",
        "total_episodes","avg_episode_runtime","poster_path"   # 🔥 Season poster_path
    ]

    df_series = df_series[SERIES_COLS].drop_duplicates(subset=["id"])
    if not df_seasons.empty:
        df_seasons = df_seasons[SEASON_COLS].drop_duplicates(subset=["series_id","season_number"])

    # 4) 시즌 기반 결측치 보완
    if not df_seasons.empty:
        df_series = fill_series_gaps(df_series, df_seasons)

    # 5) 저장
    df_series.to_csv(SERIES_CSV, index=False, encoding="utf-8-sig")
    df_seasons.to_csv(SEASONS_CSV, index=False, encoding="utf-8-sig")

    try:
        df_series.to_parquet(SERIES_PARQ, index=False)
        df_seasons.to_parquet(SEASONS_PARQ, index=False)
    except:
        pass

    for f in [SERIES_TEMP, SEASONS_TEMP]:
        if f.exists():
            f.unlink()

    elapsed = (datetime.now() - t0).total_seconds()/60

    print("="*90)
    print("🎉 수집 완료")
    print("="*90)
    print(f"📌 Series 수: {len(df_series):,}개")
    print(f"📌 Season 수: {len(df_seasons):,}개")
    print(f"📌 Series 컬럼 수: {len(SERIES_COLS)}")   # 51
    print(f"📌 Season  컬럼 수: {len(SEASON_COLS)}")  # 15
    print(f"⏱ 총 소요시간: {elapsed:.1f}분")
    print("="*90)

# ==========================================================
# RUN
# ==========================================================
if __name__ == "__main__":
    asyncio.run(main())


✅ nest_asyncio 적용 완료
🚀 TMDB FULL SERIES + SEASONS COLLECTOR (FINAL + poster_path)

📌 1단계: ID 수집


KeyboardInterrupt: 

## 수집 결과 점검